In [8]:
import requests
import lxml.html as lh
import pandas as pd
from bs4 import BeautifulSoup
from pprint import pprint
import re
import time
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary 
from selenium.webdriver.common.keys import Keys   # For keyboard keys 
from selenium.webdriver.chrome.options import Options
import datetime
import argparse
import os
from selenium.webdriver.support.ui import WebDriverWait
from urllib.parse import urlparse,urlsplit,parse_qs,parse_qsl,urlencode,urlunparse
from pathlib import Path
from selenium.common.exceptions import NoSuchElementException



In [9]:
chrome_options = Options()
path = "/home/ahmed/labs/isie/"
chrome_options.add_experimental_option("prefs", {
  "download.default_directory": path,
  "download.prompt_for_download": False,
  "download.directory_upgrade": True,
  "safebrowsing.enabled": True
})
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
locate_driver = '/usr/lib/chromium-browser/chromedriver'
driver = webdriver.Chrome(locate_driver,options=chrome_options)
#validés:
#http://www.isie.tn/resultats/resultats-presidentielles/election-presidentielle-1er-tour/ 
#http://www.isie.tn/elections/election-2011/
#http://www.isie.tn/elections/elections-municipales-2018/resultats/
#http://www.isie.tn/resultats/resultats-presidentielles/election-presidentielle-2eme-tour/
url = 'http://www.isie.tn/resultats/resultats-presidentielles/election-presidentielle-2eme-tour/'
#url_parse = urlparse(url)
driver.get(url)

In [10]:
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="bc91bf6d40f1c293d509ad86d1c5e59c")>

In [11]:
start_time = time.time()
driver.refresh();
time.sleep(1)
list_csv_link= []
#this class is the global class existed on all page 
html_list = driver.find_element_by_class_name("entry-content")
#check if buton plus exist because if existed we should click all folder to get link inside
try:
    plus_button=driver.find_element_by_class_name("treeview")
    plus_list_exist=True
except NoSuchElementException:
    print("No plus button found")
    plus_list_exist=False
links = html_list.find_elements_by_tag_name('a')
print('fetching all Files')
#loop on all link exist in html 
for num,link in enumerate(links):
    print(link.get_attribute("href"))
    #for check if buton plus exist
    if plus_list_exist:
            #for check if is link wich contain files 
        if '#' in link.get_attribute("href"):
                #print(link.text)
            link.click()
            time.sleep(0.5)            
    if link.get_attribute('href').endswith('.xlsx'):
        dicti = {'region':link.text,'link':link.get_attribute('href')}      
        list_csv_link.append(dicti)
#this because after links is opened in case button plus, we should refind all new links that appears        
if plus_list_exist:
    links = html_list.find_elements_by_tag_name('a')
    list_csv_link = [{'region':link.text,'link':link.get_attribute('href')} for link in links if link.get_attribute('href').endswith('.xlsx')]
print('we have traited ' + str(num) + ' link and files')
print("--- %s seconds ---" % (time.time() - start_time))

No plus button found
fetching all Files
http://www.isie.tn/resultats/resultats-presidentielles/election-presidentielle-2eme-tour/#
http://www.isie.tn/resultats/resultats-presidentielles/election-presidentielle-2eme-tour/#
http://www.isie.tn/wp-content/uploads/filebases/pv-auto-bv-presidentielles-tour2/Etranger/Allemagne/Allemagne.xlsx
http://www.isie.tn/resultats/resultats-presidentielles/election-presidentielle-2eme-tour/#
http://www.isie.tn/wp-content/uploads/filebases/pv-auto-bv-presidentielles-tour2/Etranger/Am%C3%A9rique%20et%20Reste%20d'Europe/AmeriqueResteEurope.xlsx
http://www.isie.tn/resultats/resultats-presidentielles/election-presidentielle-2eme-tour/#
http://www.isie.tn/wp-content/uploads/filebases/pv-auto-bv-presidentielles-tour2/Etranger/France2/France2.xlsx
http://www.isie.tn/resultats/resultats-presidentielles/election-presidentielle-2eme-tour/#
http://www.isie.tn/wp-content/uploads/filebases/pv-auto-bv-presidentielles-tour2/Etranger/Italie/Italie.xlsx
http://www.isie.t

http://www.isie.tn/wp-content/uploads/filebases/pv-auto-bv-presidentielles-tour2/Tunisie/Gafsa/Metlaoui.xlsx
http://www.isie.tn/wp-content/uploads/filebases/pv-auto-bv-presidentielles-tour2/Tunisie/Gafsa/Moulares.xlsx
http://www.isie.tn/wp-content/uploads/filebases/pv-auto-bv-presidentielles-tour2/Tunisie/Gafsa/Redeyef.xlsx
http://www.isie.tn/wp-content/uploads/filebases/pv-auto-bv-presidentielles-tour2/Tunisie/Gafsa/SidiAich.xlsx
http://www.isie.tn/wp-content/uploads/filebases/pv-auto-bv-presidentielles-tour2/Tunisie/Gafsa/Sned.xlsx
http://www.isie.tn/resultats/resultats-presidentielles/election-presidentielle-2eme-tour/#
http://www.isie.tn/wp-content/uploads/filebases/pv-auto-bv-presidentielles-tour2/Tunisie/Jendouba/AinDrahim.xlsx
http://www.isie.tn/wp-content/uploads/filebases/pv-auto-bv-presidentielles-tour2/Tunisie/Jendouba/BaltaBouAouene.xlsx
http://www.isie.tn/wp-content/uploads/filebases/pv-auto-bv-presidentielles-tour2/Tunisie/Jendouba/BouSelem.xlsx
http://www.isie.tn/wp-cont

http://www.isie.tn/wp-content/uploads/filebases/pv-auto-bv-presidentielles-tour2/Tunisie/Mannouba/ElBattan.xlsx
http://www.isie.tn/wp-content/uploads/filebases/pv-auto-bv-presidentielles-tour2/Tunisie/Mannouba/Jedaida.xlsx
http://www.isie.tn/wp-content/uploads/filebases/pv-auto-bv-presidentielles-tour2/Tunisie/Mannouba/Mannouba.xlsx
http://www.isie.tn/wp-content/uploads/filebases/pv-auto-bv-presidentielles-tour2/Tunisie/Mannouba/Mornaguia.xlsx
http://www.isie.tn/wp-content/uploads/filebases/pv-auto-bv-presidentielles-tour2/Tunisie/Mannouba/OuedEllil.xlsx
http://www.isie.tn/wp-content/uploads/filebases/pv-auto-bv-presidentielles-tour2/Tunisie/Mannouba/Tebourba.xlsx
http://www.isie.tn/resultats/resultats-presidentielles/election-presidentielle-2eme-tour/#
http://www.isie.tn/wp-content/uploads/filebases/pv-auto-bv-presidentielles-tour2/Tunisie/Medenine/BenGuerden.xlsx
http://www.isie.tn/wp-content/uploads/filebases/pv-auto-bv-presidentielles-tour2/Tunisie/Medenine/BniKhdech.xlsx
http://ww

http://www.isie.tn/wp-content/uploads/filebases/pv-auto-bv-presidentielles-tour2/Tunisie/Sidi%20Bouzid/SebelitAouledAskar.xlsx
http://www.isie.tn/wp-content/uploads/filebases/pv-auto-bv-presidentielles-tour2/Tunisie/Sidi%20Bouzid/SidiAliBenOun.xlsx
http://www.isie.tn/wp-content/uploads/filebases/pv-auto-bv-presidentielles-tour2/Tunisie/Sidi%20Bouzid/SidiBouzidEst.xlsx
http://www.isie.tn/wp-content/uploads/filebases/pv-auto-bv-presidentielles-tour2/Tunisie/Sidi%20Bouzid/SidiBouzidOuest.xlsx
http://www.isie.tn/wp-content/uploads/filebases/pv-auto-bv-presidentielles-tour2/Tunisie/Sidi%20Bouzid/SoukJedid.xlsx
http://www.isie.tn/resultats/resultats-presidentielles/election-presidentielle-2eme-tour/#
http://www.isie.tn/wp-content/uploads/filebases/pv-auto-bv-presidentielles-tour2/Tunisie/Siliana/Bargou.xlsx
http://www.isie.tn/wp-content/uploads/filebases/pv-auto-bv-presidentielles-tour2/Tunisie/Siliana/BouArada.xlsx
http://www.isie.tn/wp-content/uploads/filebases/pv-auto-bv-presidentielles-t

http://www.isie.tn/wp-content/uploads/filebases/pv-auto-bv-presidentielles-tour2/Tunisie/Zaghouan/Saouef.xlsx
http://www.isie.tn/wp-content/uploads/filebases/pv-auto-bv-presidentielles-tour2/Tunisie/Zaghouan/Zaghouan.xlsx
http://www.isie.tn/wp-content/uploads/filebases/pv-auto-bv-presidentielles-tour2/Tunisie/Zaghouan/Zriba.xlsx
http://www.isie.tn/resultats/resultats-presidentielles/election-presidentielle-2eme-tour/#
http://www.isie.tn/resultats/resultats-presidentielles/election-presidentielle-2eme-tour/#
http://www.isie.tn/resultats/resultats-presidentielles/election-presidentielle-2eme-tour/#
http://www.isie.tn/wp-content/uploads/filebases/pv-bv-presidentielles-tour2/Etranger/Allemagne/Ambassade%20de%20Tunisie%20a%20Berlin/52020200101.jpg
http://www.isie.tn/wp-content/uploads/filebases/pv-bv-presidentielles-tour2/Etranger/Allemagne/Ambassade%20de%20Tunisie%20a%20Berlin/52020200102.jpg
http://www.isie.tn/resultats/resultats-presidentielles/election-presidentielle-2eme-tour/#
http://

...

http://www.isie.tn/wp-content/uploads/filebases/pv-bv-presidentielles-tour2/Tunisie/TUNIS2/Omrane%20superieur/E.P.IBN%20ARAFA/02030402001.pdf
http://www.isie.tn/wp-content/uploads/filebases/pv-bv-presidentielles-tour2/Tunisie/TUNIS2/Omrane%20superieur/E.P.IBN%20ARAFA/02030402002.pdf
http://www.isie.tn/wp-content/uploads/filebases/pv-bv-presidentielles-tour2/Tunisie/TUNIS2/Omrane%20superieur/E.P.IBN%20ARAFA/02030402003-1-correction.pdf
http://www.isie.tn/wp-content/uploads/filebases/pv-bv-presidentielles-tour2/Tunisie/TUNIS2/Omrane%20superieur/E.P.IBN%20ARAFA/02030402003-2-correction.pdf
http://www.isie.tn/wp-content/uploads/filebases/pv-bv-presidentielles-tour2/Tunisie/TUNIS2/Omrane%20superieur/E.P.IBN%20ARAFA/02030402003.pdf
http://www.isie.tn/wp-content/uploads/filebases/pv-bv-presidentielles-tour2/Tunisie/TUNIS2/Omrane%20superieur/E.P.IBN%20ARAFA/02030402004.pdf
http://www.isie.tn/wp-content/uploads/filebases/pv-bv-presidentielles-tour2/Tunisie/TUNIS2/Omrane%20superieur/E.P.IBN%20AR

http://www.isie.tn/resultats/resultats-presidentielles/election-presidentielle-2eme-tour/#
http://www.isie.tn/wp-content/uploads/filebases/pv-bv-presidentielles-tour2/Tunisie/Zaghouan/Fahs/e.p.bent%20saaidan/1302083101.jpg
http://www.isie.tn/resultats/resultats-presidentielles/election-presidentielle-2eme-tour/#
http://www.isie.tn/wp-content/uploads/filebases/pv-bv-presidentielles-tour2/Tunisie/Zaghouan/Fahs/e.p.birmagra/13020201501.jpg
http://www.isie.tn/resultats/resultats-presidentielles/election-presidentielle-2eme-tour/#
http://www.isie.tn/wp-content/uploads/filebases/pv-bv-presidentielles-tour2/Tunisie/Zaghouan/Fahs/e.p.bouchoucha/13020101301.jpg
http://www.isie.tn/wp-content/uploads/filebases/pv-bv-presidentielles-tour2/Tunisie/Zaghouan/Fahs/e.p.bouchoucha/13020101302.jpg
http://www.isie.tn/wp-content/uploads/filebases/pv-bv-presidentielles-tour2/Tunisie/Zaghouan/Fahs/e.p.bouchoucha/13020101303.jpg
http://www.isie.tn/resultats/resultats-presidentielles/election-presidentielle-2e

http://www.isie.tn/wp-content/uploads/filebases/pv-bv-presidentielles-tour2/Tunisie/Zaghouan/Fahs/e.p%20gantaret%20fahes/13020502202.jpg
http://www.isie.tn/resultats/resultats-presidentielles/election-presidentielle-2eme-tour/#
http://www.isie.tn/wp-content/uploads/filebases/pv-bv-presidentielles-tour2/Tunisie/Zaghouan/Fahs/ep.sidi%20nawi/13020602501.jpg
http://www.isie.tn/resultats/resultats-presidentielles/election-presidentielle-2eme-tour/#
http://www.isie.tn/resultats/resultats-presidentielles/election-presidentielle-2eme-tour/#
http://www.isie.tn/wp-content/uploads/filebases/pv-bv-presidentielles-tour2/Tunisie/Zaghouan/Nadhour/e.p.2mars34/13030104401.jpg
http://www.isie.tn/wp-content/uploads/filebases/pv-bv-presidentielles-tour2/Tunisie/Zaghouan/Nadhour/e.p.2mars34/13030104402.JPG
http://www.isie.tn/wp-content/uploads/filebases/pv-bv-presidentielles-tour2/Tunisie/Zaghouan/Nadhour/e.p.2mars34/13030104403.jpg
http://www.isie.tn/wp-content/uploads/filebases/pv-bv-presidentielles-tour

http://www.isie.tn/resultats/resultats-presidentielles/election-presidentielle-2eme-tour/#
http://www.isie.tn/wp-content/uploads/filebases/pv-bv-presidentielles-tour2/Tunisie/Zaghouan/Sawaf/e.p.khathra/13060510901.jpg
http://www.isie.tn/resultats/resultats-presidentielles/election-presidentielle-2eme-tour/#
http://www.isie.tn/wp-content/uploads/filebases/pv-bv-presidentielles-tour2/Tunisie/Zaghouan/Sawaf/e.p.ouad%20twil/13060310501.jpg
http://www.isie.tn/resultats/resultats-presidentielles/election-presidentielle-2eme-tour/#
http://www.isie.tn/wp-content/uploads/filebases/pv-bv-presidentielles-tour2/Tunisie/Zaghouan/Sawaf/e.p.sawf%20gharbiya/13060210201.jpg
http://www.isie.tn/wp-content/uploads/filebases/pv-bv-presidentielles-tour2/Tunisie/Zaghouan/Sawaf/e.p.sawf%20gharbiya/13060210202.jpg
http://www.isie.tn/resultats/resultats-presidentielles/election-presidentielle-2eme-tour/#
http://www.isie.tn/wp-content/uploads/filebases/pv-bv-presidentielles-tour2/Tunisie/Zaghouan/Sawaf/e.p.sidi%

http://www.isie.tn/wp-content/uploads/filebases/pv-bv-presidentielles-tour2/Tunisie/Zaghouan/Zaghouan/e.p.oued%20rebeh/13050609302_correction11.jpg
http://www.isie.tn/wp-content/uploads/filebases/pv-bv-presidentielles-tour2/Tunisie/Zaghouan/Zaghouan/e.p.oued%20rebeh/13050609302_correction12.jpg
http://www.isie.tn/resultats/resultats-presidentielles/election-presidentielle-2eme-tour/#
http://www.isie.tn/wp-content/uploads/filebases/pv-bv-presidentielles-tour2/Tunisie/Zaghouan/Zaghouan/e.p.oued%20sbayhiya/13050508801.jpg
http://www.isie.tn/resultats/resultats-presidentielles/election-presidentielle-2eme-tour/#
http://www.isie.tn/wp-content/uploads/filebases/pv-bv-presidentielles-tour2/Tunisie/Zaghouan/Zaghouan/e.p.oued%20zit/13050809701.jpg
http://www.isie.tn/resultats/resultats-presidentielles/election-presidentielle-2eme-tour/#
http://www.isie.tn/wp-content/uploads/filebases/pv-bv-presidentielles-tour2/Tunisie/Zaghouan/Zaghouan/e.p.rwaygiya/1300408701.jpg
http://www.isie.tn/resultats/r

In [13]:
list_csv_link

[{'region': '',
  'link': 'http://www.isie.tn/wp-content/uploads/filebases/pv-auto-bv-presidentielles-tour2/Etranger/Allemagne/Allemagne.xlsx'},
 {'region': '',
  'link': "http://www.isie.tn/wp-content/uploads/filebases/pv-auto-bv-presidentielles-tour2/Etranger/Am%C3%A9rique%20et%20Reste%20d'Europe/AmeriqueResteEurope.xlsx"},
 {'region': '',
  'link': 'http://www.isie.tn/wp-content/uploads/filebases/pv-auto-bv-presidentielles-tour2/Etranger/France2/France2.xlsx'},
 {'region': '',
  'link': 'http://www.isie.tn/wp-content/uploads/filebases/pv-auto-bv-presidentielles-tour2/Etranger/Italie/Italie.xlsx'},
 {'region': '',
  'link': 'http://www.isie.tn/wp-content/uploads/filebases/pv-auto-bv-presidentielles-tour2/Etranger/Pays%20Arabes%20et%20Restes%20du%20Monde/PaysArabeEtResteDeMonde.xlsx'},
 {'region': '',
  'link': 'http://www.isie.tn/wp-content/uploads/filebases/pv-auto-bv-presidentielles-tour2/Tunisie/Ariana/ArianaVille.xlsx'},
 {'region': '',
  'link': 'http://www.isie.tn/wp-content/up

In [ ]:
len()

In [14]:
#Download part and folder structuration
base_folder = os.path.basename(os.path.dirname(url))
if not os.path.exists(path+base_folder):
    for link in list_csv_link:
        if link.get('region') !='':
            filename =  link.get('region')
        else:
            file_name_start_pos = link.get('link').rfind("/") + 1
            filename = link.get('link')[file_name_start_pos:]
            link_path_fileless = filename
        if '%' in link.get('link'):
            file_name_start =link.get('link').rfind("/") 
            file_name = link.get('link')[file_name_start:]
            link_fileless = link.get('link').replace(file_name,"")
            folder_name_start_pos =link_fileless.rfind("/") + 1
            folder_name_from_path = link_fileless[folder_name_start_pos:]
            
            if '%' in folder_name_from_path:
                sub_folder = os.path.basename(os.path.dirname(link_fileless)) 
            else:
                sub_folder = os.path.basename(os.path.basename(urlparse(link_fileless).path))  
        #sub_folder = os.path.basename(os.path.basename(urlparse(os.path.dirname(os.path.dirname(link.get('link') ))).path)) 
        else:
            sub_folder = os.path.basename(os.path.dirname( link.get('link')))
        if not os.path.exists(path+base_folder+'/'+sub_folder):
            os.makedirs(path+base_folder+'/'+sub_folder)
        r = requests.get( link.get('link'), stream=True)
        if r.status_code == requests.codes.ok:
            with open(path+base_folder +'/'+sub_folder+'/'+filename, 'wb') as f:
                for data in r:
                    f.write(data)
    print('Done.')
else:
    print('folder ' +base_folder+' already exist, maybe you have donwloaded the files .')     

folder election-presidentielle-2eme-tour already exist, maybe you have donwloaded the files .
